### AT 도매시장 통합홈페이지 _ 도매시장 실시간 경락 정보 API 
utl = https://at.agromarket.kr/openApi/price/real.do
key = 3E5842610F7D41F180751BF664323136

In [7]:
import requests
import pprint
import json
import pandas as pd
from pandas.io.json import json_normalize
import xmltodict
from bs4 import BeautifulSoup
from datetime import datetime

AT 경락 데이터 api 정보

In [8]:
key = '6afbdb611f88b65c19d0ac313cc7eac05d5fdb390ecd2de82fa14566c2fb03e4'
apiUrl = 'Grid_20161221000000000429_1'

데이터 날짜 지정

In [10]:
# 범위 시작 및 종료 날짜
start_date_string = "2017-01-01"
end_date_string = "2017-01-31"
start_date =  datetime.strptime(start_date_string,'%Y-%m-%d')
end_date = datetime.strptime(end_date_string,'%Y-%m-%d')

# 지정된 범위의 날짜를 포함하는 시리즈 생성
date_series = pd.date_range(start=start_date, end=end_date, freq='D')

# 날짜를 원하는 숫자 형식으로 변환
numeric_dates = date_series.strftime('%Y%m%d').astype(int)

# 숫자 형식의 날짜를 포함하는 데이터프레임 생성
date_df = pd.DataFrame(numeric_dates, columns=['가격등록일자'])
# 숫자 형식의 날짜를 포함하는 데이터프레임 리스트로 변경
dateList = date_df['가격등록일자'].tolist()

Target 품목 정보(홈페이지에서 품목 이름별 정보 검색 후, 정리)

In [11]:
pummok_list = [
    {'pumjongCd_list': ['080801', '080602'], 'pummok_name': '방울토마토', 'pummok_numb': 1},
    {'pumjongCd_list': ['120501'], 'pummok_name': '청양고추', 'pummok_numb': 2},
    {'pumjongCd_list': ['120601'], 'pummok_name': '꽈리고추', 'pummok_numb': 3},
    {'pumjongCd_list': ['050201'], 'pummok_name': '고구마', 'pummok_numb': 4},
    {'pumjongCd_list': ['050101', '050103'], 'pummok_name': '감자', 'pummok_numb': 5},
    {'pumjongCd_list': ['100101', '100102', '100103', '100104', '100105',], 'pummok_name': '배추', 'pummok_numb': 6},
    {'pumjongCd_list': ['100401'], 'pummok_name': '양배추', 'pummok_numb': 7},
    {'pumjongCd_list': ['100801'], 'pummok_name': '시금치', 'pummok_numb': 8},
    {'pumjongCd_list': ['100501', '100503'], 'pummok_name': '청상추', 'pummok_numb': 9},
    {'pumjongCd_list': ['100502', '100504'], 'pummok_name': '적상추', 'pummok_numb': 10},
    {'pumjongCd_list': ['090102'], 'pummok_name': '다다기오이', 'pummok_numb': 11},
    {'pumjongCd_list': ['090103'], 'pummok_name': '가시오이', 'pummok_numb': 12},
    {'pumjongCd_list': ['090101'], 'pummok_name': '취청오이', 'pummok_numb': 13},
    {'pumjongCd_list': ['120702', '120704', '120705', '120706'], 'pummok_name': '건고추', 'pummok_numb': 14},
    {'pumjongCd_list': ['090201'], 'pummok_name': '애호박', 'pummok_numb': 15},
    {'pumjongCd_list': ['090202'], 'pummok_name': '쥬키니호박', 'pummok_numb': 16},
    {'pumjongCd_list': ['080301'], 'pummok_name': '토마토', 'pummok_numb': 17},
    {'pumjongCd_list': ['110101', '110102', '110103', '110104'], 'pummok_name': '무', 'pummok_numb': 18},
    {'pumjongCd_list': ['120906'], 'pummok_name': '깐마늘', 'pummok_numb': 19},
    {'pumjongCd_list': ['120101'], 'pummok_name': '양파', 'pummok_numb': 20},
    {'pumjongCd_list': ['120201'], 'pummok_name': '대파', 'pummok_numb': 21},
    {'pumjongCd_list': ['120301'], 'pummok_name': '쪽파', 'pummok_numb': 22},
    {'pumjongCd_list': ['100901'], 'pummok_name': '미나리', 'pummok_numb': 23},
    {'pumjongCd_list': ['101101'], 'pummok_name': '깻잎', 'pummok_numb': 24},
    {'pumjongCd_list': ['132601'], 'pummok_name': '파프리카', 'pummok_numb': 25},
    {'pumjongCd_list': ['170501'], 'pummok_name': '팽이버섯', 'pummok_numb': 26},
    {'pumjongCd_list': ['171101'], 'pummok_name': '새송이버섯', 'pummok_numb': 27},
    {'pumjongCd_list': ['100201'], 'pummok_name': '얼갈이배추', 'pummok_numb': 28},
    {'pumjongCd_list': ['110301'], 'pummok_name': '당근', 'pummok_numb': 29},
]


In [12]:
# 품목별 확인을 위해 하나씩 진행해봄
pummok_list = [
    {'pumjongCd_list': ['110301'], 'pummok_name': '당근', 'pummok_numb': 29},
]

데이터 Parsing

In [13]:
import os

for pummok in pummok_list:
    
    pumjongCd_list = pummok['pumjongCd_list']
    pummok_name = pummok['pummok_name']
    pummok_numb = pummok['pummok_numb']

    print(pummok_name)

    output_folder = 'excel_result'
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_file = f'{output_folder}/pummok_{pummok_name}.csv'
    
    for date in dateList:
        
        daily_pummok_df = pd.DataFrame()
        # daily_pummok_df['datadate'] = pd.Series(dtype=object)
        daily_pummok_df = pd.DataFrame(columns=['datadate', '도매시장코드','품종코드','경락가','거래단량','거래량','산지코드','산지명']) 
        num_data = 1000

        for pumjongCd in pumjongCd_list:
            step = 0
            num_data = 1000
        
            while True:
                startIndex = step * num_data + 1
                endIndex = (step + 1) * num_data
        
                url = f'http://211.237.50.150:7080/openapi/{key}/xml/{apiUrl}/{startIndex}/{endIndex}?AUCNG_DE={date}&SPCIES_CD={pumjongCd}'
                response = requests.get(url)
                xml_data = xmltodict.parse(response.content)
                data = xml_data[apiUrl]
                data_count = int(data['totalCnt'])
            
                if data_count > 0:
                    data_list = [xml_data[apiUrl]['row']] if isinstance(xml_data[apiUrl]['row'], dict) else xml_data[apiUrl]['row']
                    daily_df = pd.DataFrame(data_list)
                    daily_df = daily_df.drop('ROW_NUM', axis=1)
                    daily_df.columns = ['datadate', '도매시장명', '도매시장코드', '법인명', '법인코드','원표번호','일련번호','경매시간','품목명','품목코드','품종명','품종코드','경락가','거래단량','규격','규격코드','등급','등급코드','산지코드','산지명','거래량']
                    daily_df = daily_df[['datadate', '도매시장코드','품종코드','경락가','거래단량','거래량','산지코드','산지명']]
                    daily_pummok_df = pd.concat([daily_pummok_df, daily_df], ignore_index=True)
                else:
                    # 빈 행이 품종별로 생기니까 하나 생기고 나면 더 안생기게.
                    if date not in daily_pummok_df['datadate'].values:
                        # totalCnt가 0인 경우 빈 데이터프레임을 생성하고 NaN 값으로
                        empty_daily_df = pd.DataFrame(columns=daily_pummok_df.columns)
                        empty_daily_df.loc[0, :] = pd.NA  # 모든 값이 결측값인 행을 추가
                        empty_daily_df['datadate'] = date
                        daily_pummok_df = pd.concat([daily_pummok_df, empty_daily_df], ignore_index=True)
                        
                if data_count <= endIndex:
                    break
                step += 1
        
        if os.path.exists(output_file):
            daily_pummok_df.to_csv(output_file, mode='a', header=False, index=False)
        else:
            daily_pummok_df.to_csv(output_file, mode='w', header=True, index=False)
        
        print(f"{pummok_name}: Saved data for {date}")


당근
당근: Saved data for 20170101
당근: Saved data for 20170102
당근: Saved data for 20170103
당근: Saved data for 20170104
당근: Saved data for 20170105
당근: Saved data for 20170106
당근: Saved data for 20170107
당근: Saved data for 20170108
당근: Saved data for 20170109
당근: Saved data for 20170110
당근: Saved data for 20170111
당근: Saved data for 20170112
당근: Saved data for 20170113
당근: Saved data for 20170114
당근: Saved data for 20170115
당근: Saved data for 20170116
당근: Saved data for 20170117
당근: Saved data for 20170118
당근: Saved data for 20170119
당근: Saved data for 20170120
당근: Saved data for 20170121
당근: Saved data for 20170122
당근: Saved data for 20170123
당근: Saved data for 20170124
당근: Saved data for 20170125
당근: Saved data for 20170126
당근: Saved data for 20170127
당근: Saved data for 20170128
당근: Saved data for 20170129
당근: Saved data for 20170130
당근: Saved data for 20170131
